# Experiment 2

This id is used as a prefix for the figure names.

In [ ]:
exp_id = 'experiment2'
from datetime import datetime
exp_id += '_' + str(datetime.now()).replace(' ', '_')

### Imports

In [ ]:
import sys
sys.path.insert(1, '../')

In [ ]:
from utils import plot_graph
from graph_loaders import load_graph
import matplotlib.pyplot as plt
import numpy as np
from approx import GBFGreedy, GBFPointSelection
from kernels import VarSpline, Diffusion
import networkx as nx
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
%load_ext autoreload
%autoreload 2
plt.rcParams.update({'font.size': 16})

### Load a graph

In [ ]:
path = '../'
# G = load_graph('sensor2', path=path)
# G = load_graph('sensor1', path=path)
# G = load_graph('emptyset', path=path)
# G = load_graph('2moon', path=path)
# G = load_graph('minnesota', path=path)
# G = load_graph('rand', path=path)
# G = load_graph('rand_sparse', path=path)
G = load_graph('bunny', path=path)

# G = nx.dorogovtsev_goltsev_mendes_graph(7)
# pos = nx.spectral_layout(G, center=[0.5, 0.5])
# nx.set_node_attributes(G, pos, 'pos')

### Define an optimization set

In [ ]:
X_train = np.arange(len(G))
y_train = np.ones(len(G))

f = lambda x: np.exp(-(4 * np.linalg.norm(x - [.5, .5], axis=1)) ** 2)
pos = np.array([[pos[0], pos[1]] for pos in nx.get_node_attributes(G, 'pos').values()])
y_train = np.array(f(pos))

### Define the common params for the point selection and optimization

In [ ]:
max_iter = 20 # Max number of point to be selected
tol_p = 1e-14  # Tolerance on the max of the squared power function
tol_f = 1e-14  # Tolerance on the residual

In [ ]:
def mean_error(y_true, y_pred):
    return np.mean(np.abs(y_true - y_pred))

def max_error(y_true, y_pred):
    return np.max(np.abs(y_true - y_pred))

def max_variance(y_true, y_pred):
    return np.max(np.abs(y_pred))

scorer = make_scorer(mean_error, greater_is_better=False)

cv = 5          # cv-fold cross validation
n_jobs = -1     # number of parallel jobs (-1: all available cores)
grid_size = 25   # size of 1d discretization grid

### Select the points for non-optimized kernels

In [ ]:
kernel = {}
kernel['var_spline'] = VarSpline(G, par=[-1, 0.01])
kernel['diffusion'] = Diffusion(G, par=[-10])

In [ ]:
non_opt_models = {}
for ker_id in kernel:
    print('Training model with ' + str(kernel[ker_id]))
    non_opt_models[ker_id] = GBFGreedy(G, kernel=kernel[ker_id], greedy_type='p_greedy', 
                      reg_par=0, 
                      max_iter=max_iter, tol_p=tol_p, tol_f=tol_f,
                      verbose=False)
    non_opt_models[ker_id].fit(X_train, y_train)

### Select the points for optimized kernels

In [ ]:
params = {}

params['var_spline'] = {
        'kernel': ['VarSpline'],
        'kernel_par': [[-x, y] for x in np.logspace(-1, 1, grid_size) for y in np.logspace(-16, 0, grid_size)]
        }

params['diffusion'] = {
        'kernel': ['Diffusion'],
        'kernel_par': [[-x] for x in np.logspace(-2, 2, grid_size)]
}

In [ ]:
opt_models = {}
for ker_id in kernel:
    print('Training optimized model with ' + str(kernel[ker_id]))
#     opt_models[ker_id] = GridSearchCV(GBFGreedy(G, greedy_type='p_greedy', 
#                                                 reg_par=0, 
#                                                 max_iter=max_iter, tol_p=tol_p, tol_f=tol_f, 
#                                                 verbose=False), 
#                                       param_grid=params[ker_id], scoring=scorer, n_jobs=n_jobs, 
#                                       cv=cv, refit=True, verbose=1)
    opt_models[ker_id] = GridSearchCV(GBFPointSelection(G, greedy_type='p_greedy', 
                                                reg_par=0, 
                                                max_iter=max_iter, tol_p=tol_p, tol_f=tol_f, 
                                                verbose=False), 
                                      param_grid=params[ker_id], scoring=scorer, n_jobs=n_jobs, 
                                      cv=cv, refit=True, verbose=1)

    opt_models[ker_id].fit(X_train, y_train)

### Check the optimal parameters

In [ ]:
print('Not optimized:')
for ker_id in kernel:
    print(non_opt_models[ker_id].kernel)
    
print('\nOptimized:')
for ker_id in kernel:
    print(opt_models[ker_id].best_estimator_.kernel)
for ker_id in kernel:
    print(opt_models[ker_id].best_estimator_.kernel.par)

### Visualize the selected points

In [ ]:
fig = plt.figure(figsize=(10, 10))
for idx, ker_id in enumerate(kernel):
    ax = plt.subplot(2, len(kernel), idx+1) 
    plot_graph(G, ax=ax, values=non_opt_models[ker_id].eval_power_fun(X_train), 
               nodelist=non_opt_models[ker_id].ctrs_, 
               cb_label='Standard deviation')
    ax.set_title(ker_id)
    
    ax = plt.subplot(2, len(kernel), len(kernel)+idx+1) 
    plot_graph(G, ax=ax, values=opt_models[ker_id].best_estimator_.eval_power_fun(X_train), 
               nodelist=opt_models[ker_id].best_estimator_.ctrs_, 
               cb_label='Standard deviation')
    ax.set_title('optimized ' + ker_id)
    
plt.savefig('figures/' + exp_id + '_points' + '.pdf', bbox_inches='tight')

### Visualize the decay of the standard deviation

In [ ]:
fig = plt.figure(figsize=(10, 5))
ax = fig.gca()
leg = []

for idx, ker_id in enumerate(kernel):
    ax.semilogy(non_opt_models[ker_id].train_hist['p'] / np.max(non_opt_models[ker_id].train_hist['p']), 'o-')
    leg.append(ker_id)
    
    ax.semilogy(opt_models[ker_id].best_estimator_.train_hist['p'] / np.max(opt_models[ker_id].best_estimator_.train_hist['p']), 'o-')
    leg.append('optimized ' + ker_id)

ax.legend(leg, fontsize=16)
ax.set_xlabel('Number of nodes', fontsize=16)
ax.set_ylabel('Max. standard deviation', fontsize=16)

for tick in ax.xaxis.get_major_ticks():
    tick.label.set_fontsize(16) 
for tick in ax.yaxis.get_major_ticks():
    tick.label.set_fontsize(16) 
ax.grid(True)

plt.savefig('figures/' + exp_id + '_p_max' + '.pdf', bbox_inches='tight')

### Visualize the decay of the residual

In [ ]:
fig = plt.figure(figsize=(7, 5))
ax = fig.gca()
leg = []

for idx, ker_id in enumerate(kernel):
    ax.semilogy(non_opt_models[ker_id].train_hist['f'])
    leg.append(ker_id)
    ax.semilogy(opt_models[ker_id].best_estimator_.train_hist['f'])
    leg.append('optimized ' + ker_id)

ax.legend(leg, fontsize=16)
ax.set_xlabel('Number of nodes', fontsize=16)
ax.set_ylabel('Max. residual', fontsize=16)

for tick in ax.xaxis.get_major_ticks():
    tick.label.set_fontsize(16) 
for tick in ax.yaxis.get_major_ticks():
    tick.label.set_fontsize(16) 
ax.grid(True)

plt.savefig('figures/' + exp_id + '_f_max' + '.pdf', bbox_inches='tight')

### Visualize the order of the first selected points

In [ ]:
import pandas as pd
data = {}

for ker_id in kernel:
    data[ker_id] = non_opt_models[ker_id].ctrs_.flatten()
    
for ker_id in kernel:
    data['optimized ' + ker_id] = opt_models[ker_id].best_estimator_.ctrs_.flatten()

points = pd.DataFrame(data)
points.head(10)